# Enhanced Rheological Analysis with True Viscosity Comparison

This notebook provides advanced rheological analysis capabilities including:
- **True viscosity comparison**: Overlay theoretical lines on experimental data
- **Enhanced dataset generation**: Comprehensive error analysis with fitted parameters
- **Interactive analysis**: Prompt-based viscosity input for validation
- **Comprehensive error metrics**: Slope and intercept errors vs. known values

## Features:
✓ Interactive true viscosity input and visualization  
✓ Enhanced dataset generation with fitting parameters and uncertainties  
✓ Error analysis comparing experimental vs. theoretical values  
✓ Comprehensive statistical analysis and visualization tools  

*Created: November 21, 2025*

# 1 - Import Required Libraries
Import all necessary libraries for enhanced rheological data analysis:
- pandas: Data manipulation and CSV handling
- numpy: Numerical operations
- matplotlib/seaborn: Visualization
- scipy: Curve fitting for rheological models

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import curve_fit
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("📚 All libraries imported successfully!")
print("🔬 Enhanced rheology analysis toolkit ready!")

# 2 - Load Datasets
Load all datasets for enhanced analysis. 
Modify the file paths and names to match your data.
The pipeline will automatically process all loaded datasets.

* Viscometer settings:
     - Brand: Brookfield
     - Model: DV2T
     - Spindle: CPE-52
* Automated data collection setup using CNC machine:
     - Brand: Genmitsu 4040-pro
     - Controller: GRBL
     - Container: 6 cells in a grid of 6x1

In [ ]:
# Define file paths for your datasets

# Si_60K datasets
file_paths = [
    'Data/Si_60K/dynamic_analysis_cell_1_20251112_104600.csv',
    'Data/Si_60K/dynamic_analysis_cell_2_20251112_104600.csv',
    'Data/Si_60K/dynamic_analysis_cell_3_20251112_104600.csv',
    'Data/Si_60K/dynamic_analysis_cell_4_20251112_104600.csv',
    'Data/Si_60K/dynamic_analysis_cell_5_20251112_104600.csv',
    'Data/Si_60K/dynamic_analysis_cell_6_20251112_104600.csv',
]

# Alternative datasets - uncomment to switch
# DOWSIL-TC-5888 datasets
# file_paths = [
#     'Data/DOWSIL_TC_5888/dynamic_analysis_cell_1_20251113_091536.csv',
#     'Data/DOWSIL_TC_5888/dynamic_analysis_cell_2_20251113_091536.csv',
#     'Data/DOWSIL_TC_5888/dynamic_analysis_cell_3_20251113_091536.csv',
#     'Data/DOWSIL_TC_5888/dynamic_analysis_cell_4_20251113_091536.csv',
#     'Data/DOWSIL_TC_5888/dynamic_analysis_cell_5_20251113_091536.csv',
#     'Data/DOWSIL_TC_5888/dynamic_analysis_cell_6_20251113_091536.csv',
# ]

# HY500 datasets
# file_paths = [
#     'Data/HY500/dynamic_analysis_cell_1_20251120_090205.csv',
#     'Data/HY500/dynamic_analysis_cell_2_20251120_090205.csv',
#     'Data/HY500/dynamic_analysis_cell_3_20251120_090205.csv',
#     'Data/HY500/dynamic_analysis_cell_4_20251120_090205.csv',
#     'Data/HY500/dynamic_analysis_cell_5_20251120_090205.csv',
#     'Data/HY500/dynamic_analysis_cell_6_20251120_090205.csv',
# ]

# Define custom names for each dataset (optional)
dataset_names = [f"Sample {i}" for i in range(1, 7)]

# Load all datasets
datasets = []
for idx, path in enumerate(file_paths):
    try:
        df = pd.read_csv(path)
        datasets.append(df)
        print(f"✓ Loaded: {dataset_names[idx]}")
    except Exception as e:
        print(f"✗ Error loading {path}: {e}")

print(f"\n📊 Total datasets loaded: {len(datasets)}")

# Preview first dataset
if datasets:
    print(f"\n🔍 Preview of {dataset_names[0]}:")
    display(datasets[0].head())
else:
    print("❌ No datasets loaded. Check file paths!")

# 3 - Define Rheometer Parameters
Set up rheometer geometry and measurement parameters.
These values are used for stress and strain calculations.
Adjust these based on your specific rheometer configuration.

In [ ]:
# Rheometer parameters
ALPHA_DEG = 3.0       # Cone half-angle in degrees
R_CM = 1.2            # Cone radius in cm
M_MAX = 7.187e-3      # Maximum torque in N·m (for 100% torque)
GAP_UM = 327          # Gap between cone tip and plate in micrometers

print("🔧 Rheometer Configuration:")
print(f"   Cone angle: {ALPHA_DEG}°")
print(f"   Cone radius: {R_CM} cm")
print(f"   Max torque: {M_MAX} N·m")
print(f"   Gap: {GAP_UM} μm")
print("✅ Parameters set successfully!")

# 4 - Core Functions: Stress & Strain Computation
Convert raw torque and speed measurements to fundamental rheological quantities:
- Shear stress (τ, Pa): Force per unit area
- Shear rate (γ̇, 1/s): Velocity gradient in the fluid

Includes finite-gap correction for cone-plate geometry.

In [ ]:
def compute_stress_strain(df, alpha_deg, R_cm, M_max, gap_um):
    """
    Compute shear stress (Pa) and shear rate (1/s) from raw measurements.
    Applies finite-gap correction for cone-plate geometry.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Dataset with height in first column, torque values in remaining columns
    alpha_deg : float
        Cone half-angle in degrees
    R_cm : float
        Cone radius in cm
    M_max : float
        Maximum torque in N·m (corresponding to 100% torque)
    gap_um : float
        Gap between cone tip and plate in micrometers
    
    Returns:
    --------
    results : dict
        {height_mm: (gamma_dot_array, tau_array)}
        gamma_dot: shear rate in 1/s
        tau: shear stress in Pa
    """
    # Convert units
    alpha = np.deg2rad(alpha_deg)  # degrees to radians
    R = R_cm / 100.0               # cm to meters
    h0 = gap_um * 1e-6             # micrometers to meters
    
    # Finite-gap correction factor
    # Accounts for deviation from ideal cone-plate geometry
    fc = 1 - (3 * h0) / (2 * R * np.tan(alpha))
    
    # Extract RPM values from column headers
    speed_labels = df.columns[1:]
    speeds = np.array([float(label.replace('RPM_', '')) for label in speed_labels])
    
    results = {}
    
    for i in range(len(df)):
        height = df.iloc[i, 0]
        torque_percent = df.iloc[i, 1:].values
        
        # Convert torque from % to N·m
        torque = (torque_percent / 100) * M_max
        
        # Calculate shear stress with finite-gap correction
        # Formula: τ = fc * (3M) / (2πR³)
        tau = fc * (3 * torque) / (2 * np.pi * R**3)  # Pa
        
        # Calculate shear rate
        # Convert RPM to angular velocity (rad/s)
        Omega = 2 * np.pi * speeds / 60  # rad/s
        # For cone-plate: γ̇ = Ω / tan(α)
        gamma_dot = Omega / np.tan(alpha)  # 1/s
        
        results[height] = (gamma_dot, tau)
    
    return results


def power_law(gamma_dot, K, n):
    """
    Power-law fluid model: τ = K·γ̇ⁿ
    
    Parameters:
    -----------
    gamma_dot : array
        Shear rate (1/s)
    K : float
        Consistency index (Pa·sⁿ)
    n : float
        Flow behavior index (dimensionless)
    
    Returns:
    --------
    tau : array
        Shear stress (Pa)
    """
    return K * gamma_dot**n


def linear_model(gamma_dot, a, b):
    """
    Linear (Newtonian) model: τ = a·γ̇ + b
    
    Parameters:
    -----------
    gamma_dot : array
        Shear rate (1/s)
    a : float
        Slope = viscosity (Pa·s)
    b : float
        Intercept (Pa), ideally close to zero
    
    Returns:
    --------
    tau : array
        Shear stress (Pa)
    """
    return a * gamma_dot + b

print("✅ Core functions defined successfully!")
print("🧮 Ready for stress-strain calculations!")

# 🆕 5 - Enhanced Function: Plot with True Viscosity Overlay
**NEW FEATURE:** Enhanced plotting function that overlays theoretical viscosity lines.
- Shows ideal τ = μ·γ̇ relationship (zero intercept)
- Interactive viscosity input for comparison
- Validates experimental fits against known material properties

In [ ]:
def plot_fitted_stress_strain_with_true_viscosity(datasets, dataset_names, alpha_deg, R_cm, 
                                                  M_max, gap_um, true_viscosity_cp=None, 
                                                  fit_method="linear"):
    """
    🆕 ENHANCED: Plot τ vs γ̇ with fitted curves and optional true viscosity line.
    
    Parameters:
    -----------
    datasets : list of pd.DataFrame
        List of datasets to analyze
    dataset_names : list of str
        Names for each dataset
    alpha_deg, R_cm, M_max, gap_um : float
        Rheometer parameters
    true_viscosity_cp : float or None
        True/known viscosity in cP. If provided, plots theoretical line τ = μ·γ̇
    fit_method : str
        "linear" or "power_law"
    """
    # Ask for true viscosity if not provided and method is linear
    if true_viscosity_cp is None and fit_method == "linear":
        while True:
            try:
                user_input = input("🔬 Enter the true/known viscosity of the Newtonian liquid in cP (or 'skip'): ")
                if user_input.lower() == 'skip':
                    true_viscosity_cp = None
                    break
                true_viscosity_cp = float(user_input)
                if true_viscosity_cp > 0:
                    break
                else:
                    print("⚠️ Please enter a positive value.")
            except ValueError:
                print("⚠️ Please enter a valid number or 'skip'.")
    
    n_datasets = len(datasets)
    ncols = 3
    nrows = (n_datasets + ncols - 1) // ncols
    
    fig, axes = plt.subplots(nrows, ncols, figsize=(15, 5 * nrows),
                            sharex=True, sharey=True)
    if n_datasets == 1:
        axes = np.array([axes])
    axes = axes.flatten()
    
    for idx, df in enumerate(datasets):
        ax = axes[idx]
        
        # Compute stress and strain for this dataset
        results = compute_stress_strain(df, alpha_deg, R_cm, M_max, gap_um)
        
        # Track all shear rate values for plotting true viscosity line
        all_gamma_dot = []
        
        for h, (gdot, tau) in results.items():
            # Filter positive values only (required for log scale)
            mask = (gdot > 0) & (tau > 0)
            gdot_fit, tau_fit = gdot[mask], tau[mask]
            
            if len(gdot_fit) < 3:  # Need at least 3 points to fit
                continue
                
            # Store shear rate values for true viscosity line
            all_gamma_dot.extend(gdot_fit)
            
            # Fit the selected model
            if fit_method == "power_law":
                model = power_law
                p0 = [0.1, 1]  # Initial guess: K=0.1, n=1
                popt, _ = curve_fit(model, gdot_fit, tau_fit, p0=p0)
                K, n = popt
                
                # Generate smooth curve for plotting
                g_line = np.logspace(np.log10(gdot_fit.min()), 
                                    np.log10(gdot_fit.max()), 100)
                tau_line = model(g_line, *popt)
                label_fit = f"Fit h={h:.2f}: τ={K:.2e}·γ̇^{n:.2f}"
            
            elif fit_method == "linear":
                model = linear_model
                p0 = [1, 0]  # Initial guess: slope=1, intercept=0
                popt, _ = curve_fit(model, gdot_fit, tau_fit, p0=p0)
                a, b = popt
                
                # Generate smooth line for plotting
                g_line = np.linspace(gdot_fit.min(), gdot_fit.max(), 100)
                tau_line = model(g_line, *popt)
                label_fit = f"Fit h={h:.2f}: τ={a:.2e}·γ̇+{b:.2e}"
            
            # Plot experimental data and fitted curve
            ax.scatter(gdot_fit, tau_fit, s=20, label=f"h={h:.2f} mm")
            ax.plot(g_line, tau_line, '--', lw=1.2, label=label_fit)
        
        # 🆕 Plot true viscosity line if provided and method is linear
        if true_viscosity_cp is not None and fit_method == "linear" and all_gamma_dot:
            # Convert viscosity from cP to Pa·s
            true_viscosity_pas = true_viscosity_cp / 1000
            
            # Create shear rate range for true viscosity line
            gamma_min = min(all_gamma_dot)
            gamma_max = max(all_gamma_dot)
            gamma_true = np.linspace(gamma_min, gamma_max, 100)
            tau_true = true_viscosity_pas * gamma_true  # τ = μ·γ̇ (zero intercept)
            
            ax.plot(gamma_true, tau_true, 'r-', lw=2.5, 
                   label=f"🎯 True: τ={true_viscosity_cp} cP·γ̇", alpha=0.8)
        
        # Set scale (log for power-law, linear for Newtonian)
        if fit_method == "power_law":
            ax.set_xscale('log')
            ax.set_yscale('log')
        
        # Formatting
        ax.set_xlabel("Shear Rate γ̇ (1/s)")
        ax.set_ylabel("Shear Stress τ (Pa)")
        title = f"{dataset_names[idx]} ({fit_method.replace('_',' ').title()})"
        if true_viscosity_cp is not None and fit_method == "linear":
            title += f"\n🎯 True μ = {true_viscosity_cp} cP"
        ax.set_title(title)
        ax.grid(True, which="both", ls="--", alpha=0.6)
        ax.legend(fontsize="x-small", frameon=False)
    
    # Hide unused subplots
    for idx in range(n_datasets, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

print("✅ Enhanced plotting function with true viscosity overlay ready!")
print("🎯 Interactive viscosity validation available!")

# 🆕 6 - Enhanced Function: Generate Dataset with Fitting Parameters
**NEW FEATURE:** Creates comprehensive datasets with fitted line parameters and error analysis.
- Extracts slope (viscosity) and intercept from fitted lines
- Calculates uncertainties from curve fitting covariance matrix
- Computes percentage errors compared to true viscosity values
- Provides detailed quality assessment metrics

In [ ]:
def generate_enhanced_dataset(datasets, dataset_names, alpha_deg, R_cm, 
                             M_max, gap_um, true_viscosity_cp=None):
    """
    🆕 ENHANCED: Generate dataset with fitting parameters and error analysis.
    
    Parameters:
    -----------
    datasets : list of pd.DataFrame
        List of datasets to analyze
    dataset_names : list of str
        Names for each dataset
    alpha_deg, R_cm, M_max, gap_um : float
        Rheometer parameters
    true_viscosity_cp : float or None
        True/known viscosity in cP for error calculation
    
    Returns:
    --------
    enhanced_datasets : list of pd.DataFrame
        Each DataFrame contains: height, slope, intercept, uncertainties, errors
    """
    # Ask for true viscosity if not provided
    if true_viscosity_cp is None:
        while True:
            try:
                user_input = input("🔬 Enter the true/known viscosity in cP (or 'skip' for no error calc): ")
                if user_input.lower() == 'skip':
                    true_viscosity_cp = None
                    break
                true_viscosity_cp = float(user_input)
                if true_viscosity_cp > 0:
                    break
                else:
                    print("⚠️ Please enter a positive value.")
            except ValueError:
                print("⚠️ Please enter a valid number or 'skip'.")
    
    enhanced_datasets = []
    
    # Convert true viscosity from cP to Pa·s for calculations
    true_viscosity_pas = true_viscosity_cp / 1000 if true_viscosity_cp is not None else None
    true_intercept = 0.0  # Theoretical intercept for Newtonian fluid is 0
    
    for idx, df in enumerate(datasets):
        print(f"\n🔬 Processing {dataset_names[idx]}...")
        
        # Compute stress and strain for this dataset
        results = compute_stress_strain(df, alpha_deg, R_cm, M_max, gap_um)
        
        # Create enhanced data for this dataset
        enhanced_data = []
        
        for h, (gdot, tau) in results.items():
            # Filter positive values only
            mask = (gdot > 0) & (tau > 0)
            gdot_fit, tau_fit = gdot[mask], tau[mask]
            
            if len(gdot_fit) < 3:  # Need at least 3 points to fit
                print(f"   ⚠️  Skipping height {h:.3f} mm (insufficient data points)")
                continue
            
            try:
                # Fit linear model: τ = a·γ̇ + b
                popt, pcov = curve_fit(linear_model, gdot_fit, tau_fit, p0=[1, 0])
                slope, intercept = popt
                
                # Calculate parameter uncertainties from covariance matrix
                param_errors = np.sqrt(np.diag(pcov))
                slope_uncertainty, intercept_uncertainty = param_errors
                
                # Calculate errors compared to true values (if available)
                if true_viscosity_pas is not None:
                    slope_error = abs(slope - true_viscosity_pas) / true_viscosity_pas * 100  # % error
                    intercept_error = abs(intercept - true_intercept)  # Absolute error in Pa
                else:
                    slope_error = np.nan
                    intercept_error = np.nan
                
                # Store the enhanced data
                enhanced_data.append({
                    'height_mm': h,
                    'slope_Pa_s': slope,
                    'intercept_Pa': intercept,
                    'slope_uncertainty_Pa_s': slope_uncertainty,
                    'intercept_uncertainty_Pa': intercept_uncertainty,
                    'slope_error_percent': slope_error,
                    'intercept_error_Pa': intercept_error,
                    'viscosity_cP': slope * 1000  # Convert to cP for convenience
                })
                
                print(f"   ✅ Height {h:.3f} mm: μ = {slope*1000:.1f} cP, intercept = {intercept:.3f} Pa")
                
            except Exception as e:
                print(f"   ❌ Error fitting height {h:.3f} mm: {e}")
                continue
        
        # Convert to DataFrame
        if enhanced_data:
            enhanced_df = pd.DataFrame(enhanced_data)
            enhanced_datasets.append(enhanced_df)
            print(f"   📊 Generated enhanced dataset with {len(enhanced_data)} valid measurements")
        else:
            enhanced_datasets.append(pd.DataFrame())  # Empty DataFrame
            print(f"   ⚠️  No valid measurements for {dataset_names[idx]}")
    
    return enhanced_datasets

print("✅ Enhanced dataset generation function ready!")
print("📊 Comprehensive fitting analysis with error metrics available!")

# 🚀 7 - Execute Enhanced Dataset Generation
Run the enhanced analysis to create comprehensive datasets with fitting parameters and error analysis.
This will generate detailed datasets containing slopes, intercepts, uncertainties, and error metrics.

In [ ]:
# Generate enhanced datasets with fitting parameters and error analysis
print("🚀 Starting enhanced dataset generation...")

# This will prompt you to enter the known viscosity for error calculation
enhanced_datasets = generate_enhanced_dataset(datasets, dataset_names, ALPHA_DEG, R_CM, 
                                             M_MAX, GAP_UM)

# Display the enhanced datasets summary
print("\n" + "="*80)
print("📊 ENHANCED DATASETS SUMMARY")
print("="*80)

for idx, (enhanced_df, name) in enumerate(zip(enhanced_datasets, dataset_names)):
    if not enhanced_df.empty:
        print(f"\n🔬 {name}:")
        print(f"   📐 Shape: {enhanced_df.shape}")
        print(f"   📋 Columns: {list(enhanced_df.columns)}")
        print(f"   📊 Mean viscosity: {enhanced_df['viscosity_cP'].mean():.1f} ± {enhanced_df['viscosity_cP'].std():.1f} cP")
        
        if not enhanced_df['slope_error_percent'].isna().all():
            print(f"   📈 Mean slope error: {enhanced_df['slope_error_percent'].mean():.2f}%")
            print(f"   📉 Mean intercept error: {enhanced_df['intercept_error_Pa'].mean():.4f} Pa")
        
        # Display first few rows
        print(f"\n   🔍 Preview:")
        display(enhanced_df.round(4))
    else:
        print(f"\n❌ {name}: No valid data")

print("\n✅ Enhanced dataset generation complete!")

# Store for further analysis
print(f"💾 Enhanced datasets stored in 'enhanced_datasets' variable")
print(f"📝 Total enhanced datasets: {len([df for df in enhanced_datasets if not df.empty])}")

# Alternative: You can also specify the true viscosity directly (example: 60000 cP)
# enhanced_datasets = generate_enhanced_dataset(datasets, dataset_names, ALPHA_DEG, R_CM, 
#                                              M_MAX, GAP_UM, true_viscosity_cp=60000)

# 🎯 8 - Execute Enhanced Visualization with True Viscosity
Run the enhanced stress-strain visualization with true viscosity overlay.
This interactive analysis allows you to compare experimental fits against known material properties.

In [ ]:
# Execute enhanced visualization with true viscosity comparison
print("🎯 Starting enhanced stress-strain visualization...")

# This will prompt you to enter the known viscosity for overlay
plot_fitted_stress_strain_with_true_viscosity(datasets, dataset_names, ALPHA_DEG, R_CM, 
                                            M_MAX, GAP_UM, fit_method="linear")

print("\n✅ Enhanced visualization complete!")
print("🔍 Red line shows theoretical τ = μ·γ̇ relationship")
print("📊 Compare experimental fits (dashed lines) vs theoretical line (solid red)")

# Alternative: You can also specify the true viscosity directly
# plot_fitted_stress_strain_with_true_viscosity(datasets, dataset_names, ALPHA_DEG, R_CM, 
#                                             M_MAX, GAP_UM, true_viscosity_cp=60000, 
#                                             fit_method="linear")

# 🎉 ENHANCED RHEOLOGY ANALYSIS COMPLETE

## 🆕 New Features Added Today:

### 1. **Interactive True Viscosity Comparison**
- `plot_fitted_stress_strain_with_true_viscosity()` function
- Overlays theoretical τ = μ·γ̇ line (red) on experimental data
- Interactive viscosity input for validation
- Perfect for validating measurements against known material properties

### 2. **Comprehensive Dataset Generation**
- `generate_enhanced_dataset()` function  
- Creates detailed datasets with:
  - ✅ Height measurements (mm)
  - ✅ Slope = viscosity (Pa·s)
  - ✅ Intercept (Pa)
  - ✅ Fitting uncertainties (from covariance matrix)
  - ✅ Slope error (% vs true viscosity)
  - ✅ Intercept error (absolute Pa vs ideal 0)

### 3. **Enhanced Error Analysis**
- Quantitative comparison vs. theoretical values
- Statistical uncertainty assessment
- Comprehensive quality metrics

## 📊 Usage Summary:

1. **Load Data**: Define file paths and load your rheometer data
2. **Set Parameters**: Configure cone-plate geometry parameters  
3. **Enhanced Analysis**: Run enhanced dataset generation
4. **Visualize**: Use true viscosity overlay plots for validation
5. **Analyze**: Review comprehensive error metrics and statistics

## 💡 Benefits:
- 🎯 **Validation**: Compare experimental vs. theoretical behavior
- 📊 **Quality Control**: Quantify measurement accuracy and precision
- 🔍 **Error Analysis**: Identify systematic biases and uncertainties
- 📈 **Comprehensive**: Full pipeline from raw data to statistical analysis

**Original `stress_strain.ipynb` remains unchanged!**  
*Enhanced functions are available in this new notebook.*